In [1]:
! pip install transformers datasets

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 298 kB 43.3 MB/s 
     |████████████████████████████████| 61 kB 499 kB/s 
     |████████████████████████████████| 3.3 MB 33.5 MB/s 
     |████████████████████████████████| 895 kB 27.7 MB/s 
     |████████████████████████████████| 596 kB 33.7 MB/s 
     |████████████████████████████████| 1.1 MB 24.3 MB/s 
     |████████████████████████████████| 243 kB 33.5 MB/s 
     |████████████████████████████████| 132 kB 33.2 MB/s 
     |████████████████████████████████| 271 kB 20.4 MB/s 
     |████████████████████████████████| 160 kB 30.1 MB/s 
     |████████████████████████████████| 192 kB 32.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading Model and Tokenizer from Hugging Face

In [3]:
MODEL_NAME = "CAMeL-Lab/bert-base-arabic-camelbert-da"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

### Loading Model with 9 Labels

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=9)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

## Data Cleaning and Pre Processing

In [ ]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/arabic_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(3)

,text,type
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture
2,\nاستنكرت إدارة المسرح الوطني التونسي الحملة ا...,culture


### Cleaning and Preprocessing

In [ ]:
ar_data = df.copy()
ar_data = ar_data.replace("diverse", "diverse news")
ar_data = ar_data.replace("culture", "diverse news")
ar_data = ar_data.replace("politic", "politics")
ar_data = ar_data.replace("technology", "tech")
ar_data = ar_data.replace("economy", "economy & business")
ar_data = ar_data.replace("internationalNews", "politics")
ar_data = ar_data[~ar_data['type'].str.contains('localnews')]
ar_data = ar_data[~ar_data['type'].str.contains('society')]

ar_data.sample(10)


,text,type
3435,\nأعلنت إدارة النادي الإفريقي عن تسمية رشيد ال...,sport
2879,\nأكدت رئيسة الحزب الدستوري الحر، عبير موسي أن...,politics
3825,\nاكدت صحيفة آس الإسبانية ان اللاعب الدولي الت...,sport
1505,\nأعلنت الجمهورية الإسلامية الإيرانية، أمس الث...,politics
3465,\nأسفرت عملية القرعة الخاصة بمقابلة الدور النه...,sport
2038,\nأظهرت بيانات حكومية أميركية رقما قياسيا جديد...,politics
2886,\nقالت حركة النهضة، في بيان أصدرته اليوم الأحد...,politics
1203,\nضرب زلزال بقوة 4.6 درجات على مقياس ريشتر، ول...,politics
3024,\nيواجه موناكو الفرنسي نظيره مانشستر سيتي بداي...,sport
434,"\nستهدفت موجة من الهجمات الالكترونية ""غير المس...",diverse news


In [ ]:
import re
import nltk
from string import punctuation
from nltk.corpus import stopwords
punctuation = punctuation + '\n'
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
def delete_links(input_text):
    pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    out_text = re.sub(pettern, ' ', input_text)
    return out_text

In [ ]:
def delete_repeated_characters(input_text):
    pattern  = r'(.)\1{2,}'
    out_text = re.sub(pattern, r"\1\1", input_text)
    return out_text

In [ ]:
def replace_letters(input_text):
    replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
    replace = dict((re.escape(k), v) for k, v in replace.items()) 
    pattern = re.compile("|".join(replace.keys()))
    out_text = pattern.sub(lambda m: replace[re.escape(m.group(0))], input_text)
    return out_text

In [ ]:
def clean_text(input_text):
    replace = r'[/(){}\[\]|@âÂ,;\?\'\"\*…؟–’،!&\+-:؛-]'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

In [ ]:
def remove_vowelization(input_text):
    vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    out_text = re.sub(vowelization, '', input_text)
    return out_text

In [ ]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [ ]:
def delete_stopwords(input_text):
    stop_words = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    wnl = nltk.WordNetLemmatizer()
    lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
    out_text = [w for w in lemmatizedTokens if not w in stop_words]
    out_text = ' '.join(out_text)
    return out_text
print(nltk.corpus.stopwords.words("arabic"))

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [ ]:
def text_prepare(input_text):
    out_text = delete_links(input_text)
    out_text = delete_repeated_characters(out_text)
    out_text = clean_text(out_text)
    #out_text = delete_stopwords(out_text)
   
    out_text = replace_letters(out_text)
    out_text = remove_vowelization(out_text)
    
    return out_text

In [ ]:
ar_data['Processed Text'] = ar_data['text'].apply(text_prepare)

ar_data.sample(10)

,text,type,Processed Text
3342,\nدارت اليوم الدفعة الثانية لمقابلات الجولة ال...,sport,دارت اليوم الدفعه الثانيه لمقابلات الجوله الثا...
691,\nأكدت وزيرة السياحة والصناعات التقليدية سلمى ...,economy & business,اكدت وزيره السياحه والصناعات التقليديه سلمى ال...
3350,\nعقدت الجامعة التونسية لكرة القدم المصغرة صبا...,sport,عقدت الجامعه التونسيه لكره القدم المصغره صباح ...
1506,\nهاجم شخص مسلح عدد من المارة بسكين كان يحملها...,politics,هاجم شخص مسلح عدد من الماره بسكين كان يحملها ف...
326,\nأعلنت متحدثة باسم حديقة الحيوان في مدينة أوس...,diverse news,اعلنت متحدثه باسم حديقه الحيوان في مدينه اوسنا...
3816,\nتمكن الملعب القابسي من الفوز على مستقبل المر...,sport,تمكن الملعب القابسي من الفوز على مستقبل المرسى...
785,\nبلغت نسبة التضخم عند الإستهلاك العائلي لسنة ...,economy & business,بلغت نسبه التضخم عند الاستهلاك العائلي لسنه حو...
2485,\nأعلن رئيس الحكومة الأسبق مهدي جمعة، خلال ندو...,politics,اعلن رئيس الحكومه الاسبق مهدي جمعه خلال ندوه ص...
1477,\nقدّرت ثروة الرئيس الأمريكي باراك أوباما،وفق ...,politics,ثروه الرئيس الامريكي باراك اوباما وفق ما اظهرت...
2623,\nجدّد المكتب التنفيذي لحركة النهضة في اجتماعه...,politics,المكتب التنفيذي لحركه النهضه في اجتماعه مساء ا...


In [ ]:
from sklearn.preprocessing import LabelEncoder
ar_label_encoder = LabelEncoder()
ar_data['Category Encoded'] = ar_label_encoder.fit_transform(ar_data['type'])

ar_data.sample(10)

,text,type,Processed Text,Category Encoded
1504,\nبعد جدل من أيام، حول الصورة التي التُقطت لها...,politics,بعد جدل من ايام حول الصوره التي لها جوار ضحايا...,2
3428,\nظهر اللاعب السابق للنادي الإفريقي بسام الصرا...,sport,ظهر اللاعب السابق للنادي الافريقي بسام الصرارف...,3
3594,\nتأهل اللاعب التونسي مالك الجزيري المصنف 53 ع...,sport,تاهل اللاعب التونسي مالك الجزيري المصنف الى ال...,3
3046,\nتم اليوم غلق ملعب 15 أكتوبر ببنزرت حيث إنطلق...,sport,تم اليوم غلق ملعب اكتوبر ببنزرت حيث انطلقت اشغ...,3
3988,\nيواجه لاعب التنس مالك الجزيري المصنف 47 عالم...,sport,يواجه لاعب التنس مالك الجزيري المصنف عالميا ال...,3
3978,\nتحتضن مدينة المنستير الدورة الدولية للتنس لل...,sport,تحتضن مدينه المنستير الدوره الدوليه للتنس للاو...,3
144,\nتوصلت أبحاث جديدة إلى اثبات حدوث مضار كبيرة ...,diverse news,توصلت ابحاث جديده الى اثبات حدوث مضار كبيره لل...,0
701,\nبلغ معدل التضخّم 4.6 بالمائة في شهر جانفي ال...,economy & business,بلغ معدل بالمائه في شهر جانفي الماضي ومقارنه ب...,1
2056,\nرفض الرئيس السوري بشّار الأسد السماح لوفود ت...,politics,رفض الرئيس السوري الاسد السماح لوفود تابعه لمن...,2
3951,\nينزل فريق أرسنال ضيفا على نادي ساتون يونايتد...,sport,ينزل فريق ارسنال ضيفا على نادي ساتون يونايتد ا...,3


In [ ]:
ar_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4463 entries, 0 to 4462
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text              4463 non-null   object
 1   type              4463 non-null   object
 2   Processed Text    4463 non-null   object
 3   Category Encoded  4463 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 174.3+ KB


## Splitting

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(ar_data[['Processed Text','Category Encoded']], test_size = 0.2, stratify=ar_data['Category Encoded'])

In [ ]:
df_train['Category Encoded'].value_counts()

2    1598
3    1165
0     479
1     262
4      66
Name: Category Encoded, dtype: int64

In [ ]:
df_train['text'] = df_train['Processed Text']
df_test['text'] = df_test['Processed Text']

df_train['label'] = df_train['Category Encoded']
df_test['label'] = df_test['Category Encoded']




df_train = df_train.drop(columns=['Category Encoded'])
df_test = df_test.drop(columns=['Category Encoded'])
df_train = df_train.drop(columns=['Processed Text'])
df_test = df_test.drop(columns=['Processed Text'])



from datasets import Dataset
df_train_dataset = Dataset.from_pandas(df_train)
df_test_dataset = Dataset.from_pandas(df_test)

In [ ]:
df_test

,text,label
2349,ناقشت لجنه الصحه والشؤون الاجتماعيه صلب مجلس ن...,2
4030,تدور اليوم الدفعه الثانيه للدور الربع النهائي ...,3
842,اصبح الاقبال المتزايد على التكييف يستاثر خلال ...,1
279,قام متجر الماني مؤخرا بنشر مقطع فيديو اعلاني ع...,0
541,اعلنت السلطات الاسبانيه عن القاء القبض على افر...,0
...,...,...
2837,اكد ياسين ابراهيم رئيس حزب افاق تونس انه حزبه ...,2
2812,اعلنت الهيئه السياسيه لحزب حركه نداء تونس عن ا...,2
317,اعلنت السلطات الاوكرانيه عن نفوق عشرات الدلافي...,0
3763,الارجنتيني دييغو سيميوني مدرب اتلتيكو مدريد ال...,3


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length = 512)

tokenized_train_dataset = df_train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = df_test_dataset.map(tokenize_function, batched=True)


# def tokenize_function(examples):
#     return tokenizer(examples, padding="max_length", truncation=True, max_length=512)

# tokenized_text = df['text'].apply(tokenize_function)




  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['__index_level_0__'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['__index_level_0__'])


In [ ]:
from transformers import TrainingArguments
#Directory to save chkpoints
# training_args = TrainingArguments("drive/MyDrive/nlp_fine_tuning_model/test_trainer", num_train_epochs=5)


training_args = TrainingArguments(
    output_dir='drive/MyDrive/nlp_fine_tuning_model/test_trainer',           # output directory
    evaluation_strategy="epoch",                                             # Evaluation is done at the end of each epoch.
    num_train_epochs=3,                                                      # total number of training epochs
    # per_device_train_batch_size=16,                                          # batch size per device during training
    # per_device_eval_batch_size=16,                                           # batch size for evaluation
    warmup_steps=500,                                                        # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                                       # strength of weight decay
    save_total_limit=1,                                                      # limit the total amount of checkpoints. Deletes the older checkpoints.    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_train_dataset, eval_dataset=tokenized_test_dataset)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 3570
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1341


Epoch,Training Loss,Validation Loss
1,No log,0.481415
2,0.606300,0.288362
3,0.278100,0.312268


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 893
  Batch size = 8
Saving model checkpoint to drive/MyDrive/nlp_fine_tuning_model/test_trainer/checkpoint-500
Configuration saved in drive/MyDrive/nlp_fine_tuning_model/test_trainer/checkpoint-500/config.json
Model weights saved in drive/MyDrive/nlp_fine_tuning_model/test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 893
  Batch size = 8
Saving model checkpoint to drive/MyDrive/nlp_fine_tuning_model/test_trainer/checkpoint-1000
Configuration saved in drive/MyDrive/nlp_fine_tuning_model/test_trainer/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/nlp_fine_tuning_model/test_train

TrainOutput(global_step=1341, training_loss=0.35768120898674177, metrics={'train_runtime': 2265.0218, 'train_samples_per_second': 4.728, 'train_steps_per_second': 0.592, 'total_flos': 2818096509634560.0, 'train_loss': 0.35768120898674177, 'epoch': 3.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 893
  Batch size = 8


{'eval_accuracy': 0.9339305711086227,
 'eval_loss': 0.3122679591178894,
 'eval_runtime': 63.9254,
 'eval_samples_per_second': 13.969,
 'eval_steps_per_second': 1.752}

In [ ]:
def classify_news(texts):
    test_df = pd.DataFrame({'text':texts})
    test_dataset = Dataset.from_pandas(test_df)    
    tokenized = test_dataset.map(tokenize_function, batched=True)
    predictions = trainer.predict(tokenized)
    preds = np.argmax(predictions.predictions, axis=-1)
    preds = ar_label_encoder.inverse_transform(preds)
    for i in range(len(texts)):
      print('The Type of {} is:\n        {}'.format(texts[i],preds[i]))

In [ ]:
text = ['الدولار ينخفض بسرعة كبيرة جدا', 'محمد صلاح يحرز هدفا في الدوري الانجليزي', 'اغنية انت الحظ تتصدر اليوتيوب']
classify_news(text)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 3
  Batch size = 8


The Type of الدولار ينخفض بسرعة كبيرة جدا is:
        economy & business
The Type of محمد صلاح يحرز هدفا في الدوري الانجليزي is:
        sport
The Type of اغنية انت الحظ تتصدر اليوتيوب is:
        diverse news


## Saving the model

In [ ]:
dir = 'drive/MyDrive/nlp_fine_tuning_model'

tokenizer.save_pretrained(dir)
model.save_pretrained(dir)

tokenizer config file saved in drive/MyDrive/nlp_fine_tuning_model/tokenizer_config.json
Special tokens file saved in drive/MyDrive/nlp_fine_tuning_model/special_tokens_map.json
Configuration saved in drive/MyDrive/nlp_fine_tuning_model/config.json
Model weights saved in drive/MyDrive/nlp_fine_tuning_model/pytorch_model.bin


## Loading Model

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(dir)
model = AutoModelForSequenceClassification.from_pretrained(dir,  local_files_only=True)

from transformers import TrainingArguments
#Directory to save chkpoints
# training_args = TrainingArguments("test_trainer_train2")

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_train_dataset, eval_dataset=tokenized_test_dataset)

Didn't find file drive/MyDrive/nlp_fine_tuning_model/added_tokens.json. We won't load it.
loading file drive/MyDrive/nlp_fine_tuning_model/vocab.txt
loading file drive/MyDrive/nlp_fine_tuning_model/tokenizer.json
loading file None
loading file drive/MyDrive/nlp_fine_tuning_model/special_tokens_map.json
loading file drive/MyDrive/nlp_fine_tuning_model/tokenizer_config.json
loading configuration file drive/MyDrive/nlp_fine_tuning_model/config.json
Model config BertConfig {
  "_name_or_path": "drive/MyDrive/nlp_fine_tuning_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0

In [ ]:
def classify_news(texts):
    test_df = pd.DataFrame({'text':texts})
    test_dataset = Dataset.from_pandas(test_df)    
    tokenized = test_dataset.map(tokenize_function, batched=True)
    predictions = trainer.predict(tokenized)
    preds = np.argmax(predictions.predictions, axis=-1)
    preds = ar_label_encoder.inverse_transform(preds)
    for i in range(len(texts)):
      print('The Type of {} is:\n        {}'.format(texts[i],preds[i]))

In [ ]:
text = ['الدولار ينخفض بسرعة كبيرة جدا', 'محمد صلاح يحرز هدفا في الدوري الانجليزي', 'اغنية انت الحظ تتصدر اليوتيوب']
classify_news(text)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 3
  Batch size = 8


The Type of الدولار ينخفض بسرعة كبيرة جدا is:
        economy & business
The Type of محمد صلاح يحرز هدفا في الدوري الانجليزي is:
        sport
The Type of اغنية انت الحظ تتصدر اليوتيوب is:
        diverse news
